# Convolutional neural network
Convolutional neural network (later in notebook as *CNN*) owes it's name after convolution blocks. Today I will explain you how does convolution block work and how to use them in neural network in order to create *CNN*.
***
Table of contents
 - convolution block
 - max pooling
 - dropout regularization
 - exercise



## Setup

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.datasets import mnist
from keras.utils import np_utils

import matplotlib.pyplot as plt

## Convolution block

![](images/kernel.gif)

![](images/conv.gif)

$ \
\hat{y}(i, j) = \omega * f(i,j) = \sum_{da=-a}^{a} \sum_{db=-b}^b \omega(da, db)f(i + da, j + db)\\
\omega \to kernel \\
\omega_{{a}\times{b}} = \begin{bmatrix} \omega_{1,1} & ... & \omega_{1,b} \\ . & . & . \\ \omega_{a,1} & . & \omega_{a,b} \end{bmatrix}\
$  

In convolution block we take vector of kernels.

![](images/conv_bl.gif)






Tensorflow

In [ ]:
input_shape=(28, 28, 1)
kernel_size=(3, 3)
kernel_num = 32
layers.Conv2D(kernel_num, kernel_size, activation='relu', input_shape=input_shape)

Pytorch

In [ ]:
in_channels = 3
out_channels = 8
kernel_size = 2
nn.Conv2d(in_channels, out_channels, kernel_size)

## Max pooling

![](images/pooling.gif)

Here we move our kernel just like in convolution block but this time we only take max value of kernel.

Tensorflow

In [ ]:
pool_size = (2, 2)
m = layers.MaxPooling2D(pool_size)
inp = tf.constant([[0, 0, 1, 1],
                  [1, 4, 5, 3],
                  [6, 0, 2, 4],
                  [0, 4, 3, 10]])
print(inp)
print("-" * 50)
print(f"Max pooling {pool_size}")
inp = tf.reshape(inp, (1, 4, 4, 1))
out = m(inp)
out = tf.reshape(out, (2, 2))
print(out)

Pytorch

In [ ]:
m = nn.MaxPool2d(pool_size)

inp = torch.randn(1, 4, 4)
print(inp.shape)
print(inp[0])
out = m(inp)
print("-" * 50)
print(f"Max pooling {pool_size}")
print(out.shape)
print(out[0])

## Exercise

This time we will be using *MNIST* dataset. Next time we will be using something more challenging

I want you to complete implementation of convolutional neural networks.
You can choose two frameworks to implement your solution *tensorflow* or *pytorch*.

Tensorflow

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.reshape(-1, 28, 28, 1), x_test.reshape(-1, 28, 28, 1)
y_train, y_test = np_utils.to_categorical(y_train), np_utils.to_categorical(y_test)

input_shape = (28, 28, 1)
num_classes = 10

model = models.Sequential([
    # Complete your model implementation
    # Remember about shapes!!!
    
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

optimizer = 'adam'
loss = losses.CategoricalCrossentropy(from_logits=True)
metrics=['accuracy']
epochs = 2
batch_size = 8

model.compile(optimizer, loss, metrics)

model.fit(x_train, y_train,
          epochs=epochs,
          batch_size=batch_size)

In [ ]:
img = x_test[0]
img = img.reshape(1, 28, 28, 1)

plt.imshow(img[0], cmap='gray')
plt.axis('off')
plt.show()

model.predict(img)[0].argmax()

Pytorch

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.reshape(-1, 1, 28, 28), x_test.reshape(-1, 1, 28, 28)

num_classes = 10
batch_size = 4
epochs = 2

def data_generator(x, y):
    for i in range(0, len(x), batch_size):
        yield torch.from_numpy(x[i:i+batch_size%len(x)]).float(), torch.from_numpy(y[i:i+batch_size%len(x)]).long()
    

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Complete model implementation
        # Remeber about shapes! You may use below example
        # self.conv1 = nn.Conv2d(inp_channels, out_channels, kernel_size)
        # self.pool = nn.MaxPool2d(pool_size)
        # self.conv2 = nn.Conv2d(inp_channels, out_channels, kernel_size)
        # self.fc1 = nn.Linear(last_size * last_kernel_size * last_kernel_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = x.view(-1, last_size * last_kernel_size * last_kernel_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

for epoch in range(epochs):
    
    loss = 0.0
    step = 1
    for batch, labels in data_generator(x_train, y_train):
        
        optimizer.zero_grad()
        
        outputs = net(batch)
        batch_loss = criterion(outputs, labels) 
        batch_loss.backward()
        optimizer.step()
        
        loss += batch_loss.item()
        if step % 1000 == 0:
            print(f"({epoch}, {step * batch_size}) loss: {loss:.3f}")
            loss = 0
            
        step += 1


In [ ]:
img = x_test[0]

plt.imshow(img.reshape(1, 28, 28, 1)[0], cmap='gray')
plt.axis('off')
plt.show()

img = img.reshape(1, 1, 28, 28)
net(torch.from_numpy(img).float()).detach().numpy().argmax()